In [3]:
import tensorflow as tf
# from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split

import cv2
import os
import numpy as np

In [13]:
# Define input shape based on your image size (here, 640x360)
input_shape = (360, 640, 3)

# Load MobileNet, excluding the final classification layer
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)


x = GlobalAveragePooling2D()(mobilenet.output)

predictions = Dense(2, activation='sigmoid')(x)

model = tf.keras.Model(inputs=mobilenet.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [4]:
###default

X = []
y = []
folder_path = 'classification_data/'

for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder_full_path, filename))

            X.append(img)
            
            y.append(class_label)
        

In [5]:
## aug
folder_path = 'augmented_data/'

for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg") :
            img = cv2.imread(os.path.join(folder_full_path, filename))
        
            # Append image to X and label to y
            X.append(img)
            
            
            y.append(class_label)

In [6]:
for n in range(len(X)):
    if n % 10 == 0:
        X[n] = cv2.cvtColor(X[n], cv2.COLOR_BGR2HSV)

In [7]:
X = np.asarray(X)
y = np.asarray(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)



(1864, 360, 640, 3) (1864, 2)


In [8]:
model_rap = tf.keras.models.load_model('classificator_newgen_augment.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=2, validation_data=(X_val, y_val))
model_rap.save('classificator_newgen_augment2.h5')

Epoch 1/2
59/59 ━━━━━━━━━━━━━━━━━━━━ 191s 3s/step - accuracy: 0.8245 - loss: 0.3872 - val_accuracy: 0.8415 - val_loss: 0.3837
Epoch 2/2
59/59 ━━━━━━━━━━━━━━━━━━━━ 183s 3s/step - accuracy: 0.8467 - loss: 0.3630 - val_accuracy: 0.8009 - val_loss: 0.3937


In [1]:
model.fit(X_train, y_train, epochs=2, validation_data=(X_val, y_val))


NameError: name 'model' is not defined

In [19]:
model.save('classificator_newgen_augment.h5')

In [9]:
model_rap = tf.keras.models.load_model('classificator_newgen_augment2.h5')


for n in range(20):
    y_real = y_val[n]
    tbp = np.asarray(X_val[n])
    cv2.imshow('ss', tbp)
    tbp = np.expand_dims(tbp, axis=0)
    predictions = model_rap.predict(tbp)
    print(predictions, "\n", y_real)

    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.73107713 0.1901052 ]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[0.5325008  0.15768382]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
[[0.04249566 0.6500043 ]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
[[0.56193626 0.2174966 ]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[0.3195008  0.41426015]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[0.8333505  0.20539805]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
[[0.32585075 0.49869403]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
[[0.6378327  0.32990855]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
[[0.6095849 0.5014807]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
[[0.5882802  0.29877174]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
[[0.848505   0.10922904]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[[0.49784544 0.38235372]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
[[0.3201364  0.51327956]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
[[0.87436616 0.06

augmented data fragment

In [5]:
model_rap = tf.keras.models.load_model('classificator_6_augmented.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
model_rap.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
model_rap.save('classificator_6_augmented_hsv.h5')

Epoch 1/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.6969 - loss: 0.6479 - val_accuracy: 0.7436 - val_loss: 0.5233
Epoch 2/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.7596 - loss: 0.4912 - val_accuracy: 0.7543 - val_loss: 0.4926
Epoch 3/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.7682 - loss: 0.4717 - val_accuracy: 0.7436 - val_loss: 0.4929
Epoch 4/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.7594 - loss: 0.4907 - val_accuracy: 0.7286 - val_loss: 0.4974
Epoch 5/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.7698 - loss: 0.4797 - val_accuracy: 0.7350 - val_loss: 0.4821
